In [ ]:
import pydrake

# 背景

许多工程问题可以表述为数学优化问题, 一般的数学优化问题可以表示为

$$
\begin{aligned}
\begin{array}{rl}
    \min_x             \;   &  f(x) \\
    \text{subject to}  \;   &  x \in\mathcal{S}
\end{array}
\qquad
\boxed{
    \begin{array}{ll}
        \text{The real-valued decision variable is}                         &x\\
        \text{The real-valued cost function is}                             &f(x)\\
        \text{The constraint set is}                                        &\mathcal{S}\\
        \text{The optimal } x \text{ that minimizes the cost function is}   &x^*
    \end{array}
}
\end{aligned}
$$

比如下面这个优化问题

$$
\begin{aligned}
\begin{array}{rl}
    \min_x              \;  & x^3 + 2x + 1\\
    \text{subject to}\; \;  & x \ge 1
\end{array}
\quad
\boxed{
    \begin{array}{ll}
        \text{The real-valued decision variable is}             &  x\\
        \text{The real-valued cost function }f(x) \text{ is}    &  x^3 + 2x + 1\\
        \text{The set }\mathcal{S} \text{ of constraints is}    &  x \ge 1\\
        \text{The value that minimizes the cost function is}    &  x^* = 1
    \end{array}
}
\end{aligned}
$$

一般来说优化问题有很多分类, 比如线性规划、二次规划、混合整数规划等, 不同种类的优化问题有不同的求解方法. 每类优化问题都有多种求解器, 但每个求解器有自己的 API 和数据结构. 通常, 用户从一个求解器切换到另一个时需要重写代码. 为了解决这个问题, `Drake` 通过 `MathematicalProgram` 类提供了一个公共 API. 除此之外, 约束和成本函数可以用符号变量编写代码. `Drake` 的 `MathematicalProgram` 类似于 MATLAB 中的 [YALMIP](https://yalmip.github.io/) 或 Julia 中的 [JuMP](https://github.com/JuliaOpt/JuMP.jl). `drake` 支持 [多种求解器](https://drake.mit.edu/doxygen_cxx/group__solvers.html), 有的是开源的, 有的需要许可证. `drake` 能求解的优化问题包括: 

1. 线性规划
2. 二次规划
3. 二阶锥规划
4. 非线性非凸规划
5. 半定规划
6. 平方和规划
7. 混合整数规划(混合整数线性规划、混合整数二次规划、混合整型二次锥规划). 
8. 线性互补问题

这里仅演示基本用法

# `MathematicalProgram` 类

## 初始化 `MathematicalProgram`

In [ ]:
from pydrake.solvers import MathematicalProgram
import numpy as np
import matplotlib.pyplot as plt

# 创建一个空的 MathematicalProgram 实例: 没有决策变量, 约束和代价函数
prog = MathematicalProgram()


## 创建决策变量

In [ ]:
# 添加决策变量: 添加两个连续的决策变量, x 是一个 numpy 数组. NewBinaryVariables 创建的是 0-1 决策变量
x = prog.NewContinuousVariables(2)
# 决策变量 x 默认的名字叫 "x(0)" 和 "x(1)"
print(x)
print(1 + 2 * x[0] + 3 * x[1] + 4* x[1])

In [ ]:
# 也可以自定义决策变量的名字
y = prog.NewContinuousVariables(2, 'dog')
print(y)
print(y[0] + y[0] + y[1] * y[1] * y[1])

In [ ]:
# 创建一个 3x2 的矩阵, 命名为 A
var_matrix = prog.NewContinuousVariables(3, 2, 'A')
print(var_matrix)

## 添加约束

这里仅演示基本用法. drake 会自动判断约束类型

In [ ]:
# 等式约束
prog.AddConstraint(x[0] * x[1] == 1)
# 不等式约束
prog.AddConstraint(x[0] >= 0)
prog.AddConstraint(x[0] - x[1] <= 0)

## 添加代价函数

drake 会自动判断约束的类型

In [ ]:
# 添加代价函数: x[0] ** 2 + x[0] + x[1] + 3. 故意分两次添加
prog.AddCost(x[0] ** 2 + 3)  # 二次函数
prog.AddCost(x[0] + x[1])  # 线性函数

## 优化问题求解

drake 会自动判断优化问题的类型并调用最合适的求解器

In [ ]:
"""
min x(0)^2 + x(1)^2
s.t. x(0) + x(1) = 1
           x(0) <= x(1)
"""
from pydrake.solvers import Solve

# 构建优化问题
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] + x[1] == 1)
prog.AddConstraint(x[0] <= x[1])
prog.AddCost(x[0] ** 2 + x[1] ** 2)

# 求解优化问题
result = Solve(prog)

# 求解结果
print("Success? ", result.is_success())
# 最优解
print('x* = ', result.GetSolution(x))
# 最有代价
print('optimal cost = ', result.get_optimal_cost())
# 用的是什么求解器
print('solver is: ', result.get_solver_id().name())

有时候优化问题没有解

In [ ]:
# 优化问题无解
prog = MathematicalProgram()
x = prog.NewContinuousVariables(1)[0]
y = prog.NewContinuousVariables(1)[0]
prog.AddConstraint(x + y >= 1)
prog.AddConstraint(x + y <= 0)
prog.AddCost(x)

result = Solve(prog)
print("Success? ", result.is_success())
print(result.get_solution_result())

## 手动选择求解器

如果不想 `drake` 自动选择求解器, 我们可以显式地实例化一个求解器, 并调用它的 `Solve` 函数. 有两种方法可以实例化求解器. 例如, 如果我想使用开源求解器 `IPOPT`, 有以下两种方法: 

1. 最简单的方法: `solver = IpoptSolver()`
2. `solver = MakeSolver(IpoptSolver().solver_id())`

In [ ]:
"""
min x(0)
s.t x(0) + x(1) = 1
    0 <= x(1) <= 1
"""

from pydrake.solvers import IpoptSolver

prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] + x[1] == 1)
prog.AddConstraint(0 <= x[1])
prog.AddConstraint(x[1] <= 1)
prog.AddCost(x[0])

solver = IpoptSolver()

# 起始值设为 [1, 1], 第三个参数是 IpoptSolver 的选项
result = solver.Solve(prog, np.array([1, 1]), None)

print(result.get_solution_result())
print("x* = ", result.GetSolution(x))
print("Solver is ", result.get_solver_id().name())
print("Ipopt solver status: ", result.get_solver_details().status,
      ", meaning ", result.get_solver_details().ConvertStatusToString())

In [ ]:
from pydrake.solvers import MakeSolver
solver = MakeSolver(IpoptSolver().solver_id())
result = solver.Solve(prog)
print(result.get_solution_result())
print("x* = ", result.GetSolution(x))

每个求解器都记录了自身的详细信息. 对于 `result.get_solver_details()` 的返回参数中存储的内容, 应该用 `FooSolverDetails` 类. 例如, 如果知道调用了 IPOPT, 那么参考 `IpoptSolverDetails` 类；如果是 OSQP 求解器, 就参考 `OsqpSolverDetails`. 

In [ ]:
print("Ipopt solver status: ", result.get_solver_details().status,
      ", meaning ", result.get_solver_details().ConvertStatusToString())

## 优化问题的初始解

一些优化问题, 如非线性优化, 需要初始解. 其他类型的问题, 如二次规划、混合整数优化等, 如果提供良好的初始解, 可以更快地求解问题. Solve 函数中可以把初始解作为输入参数, 否则 Drake 将使用零值向量作为初始解. 比如下面这个问题初始解问题的结果影响很大, 如果没有用户提供的初始解, 求解器可能无法找到最优解. 更多初始解, 参考 nonlinear_program.ipynb

In [ ]:
from pydrake.solvers import IpoptSolver
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0]**2 + x[1]**2 == 100.)
prog.AddCost(x[0]**2-x[1]**2)
solver = IpoptSolver()
# The user doesn't provide an initial guess.
result = solver.Solve(prog, None, None)
print(f"Without a good initial guess, the result is {result.is_success()}")
print(f"solution {result.GetSolution(x)}")
# Pass an initial guess
result = solver.Solve(prog, [-5., 0.], None)
print(f"With a good initial guess, the result is {result.is_success()}")
print(f"solution {result.GetSolution(x)}")

# 回调函数

In [ ]:
fig = plt.figure()
curve_x = np.linspace(1, 10, 100)
ax = plt.gca()
ax.plot(curve_x, 9./curve_x)
ax.plot(-curve_x, -9./curve_x)
ax.plot(0, 0, 'o')
x_init = [4., 5.]
ax.plot(x_init[0], x_init[1], 'x', color='red')

def update(x):
    ax.plot(x[0], x[1], 'x', color='red')
    
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] * x[1] == 9)
prog.AddCost(x[0]**2 + x[1]**2)
prog.AddVisualizationCallback(update, x)
result = Solve(prog, x_init)
print(f"Success? {result.is_success()}")
print(f"solution {result.GetSolution(x)}")